In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. データ読み込み
ratings = pd.read_csv("ratings.csv") # ここのシステムを考え直す（評価が変わるたびにcsvをダウンロードするわけにはいかない）
movies  = pd.read_csv("tmdb_5000_movies.csv") #　ここも、新しい映画が公開されるたびにダウンロードするわけにはいかない

# 1.1 列名確認（moviesがどんなカラムを持っているかをチェック）
print("columns:", movies.columns)

# 2. ID 列が 'id' なら 'movieId' にリネーム
if "id" in movies.columns:
    movies = movies.rename(columns={"id": "movieId"})
movies["movieId"] = pd.to_numeric(movies["movieId"], errors="coerce")

# 3. NaN を空文字に置き換え
movies["overview"] = movies["overview"].fillna("")

# 4. TF–IDF ベクトル化
vec = TfidfVectorizer(
    stop_words="english", 
    max_features=5000,
    )
tfidf_matrix = vec.fit_transform(movies["overview"].tolist()) # 映画ごとのあらすじを（映画数✖️5000）の行列に変換し、単語の重要度を計算

# 5. 好きな映画ID 抽出（★4以上）
liked = ratings[ratings["rating"] >= 4]["movieId"].tolist()
liked_indices = movies[movies["movieId"].isin(liked)].index.tolist() # 映画データの表の何行目（index）に該当するかを探し出す

# 6. ユーザーベクトル
user_vec = tfidf_matrix[liked_indices].mean(axis=0)
print(user_vec.shape)
user_vec = np.asarray(user_vec)
print(user_vec)

# 7. 類似度計算
sims = cosine_similarity(user_vec, tfidf_matrix).flatten()
for idx in liked_indices:
    sims[idx] = -1

# 8. 上位5件表示
top5_idx = np.argsort(sims)[::-1][:5]
recommendations = movies.iloc[top5_idx][["movieId","title"]]
display(recommendations)


columns: Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')
(1, 5000)
[[0. 0. 0. ... 0. 0. 0.]]


,movieId,title
3337,238,The Godfather
867,242,The Godfather: Part III
2942,304357,Woman in Gold
4531,30246,Civil Brand
3285,153397,Restless
